In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
mp_holistic = mp.solutions.holistic              # mediapipe 姿態偵測
mp_drawing = mp.solutions.drawing_utils          # mediapipe 繪圖方法

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
   # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
   # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )  


In [ ]:
# KeyPoint如何連接
# mp_holistic.POSE_CONNECTIONS

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        # Make detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        #Draw Landmarks
        draw_styled_landmarks(image,results)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
len(results.face_landmarks.landmark)

In [ ]:
draw_styled_landmarks(frame,results)

In [ ]:
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

In [ ]:
results.pose_landmarks.landmark[0] #.x .y .z .visibility
#提取第一個frame的landmark的各個數值

In [ ]:
pose=[]
for res in results.pose_landmarks.landmark:#存成array
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
# 33組x,y,z,visibility攤平成一個大陣列有132個landmark
# pose.shape (33,4)攤平成(132,)
pose=np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(132)
face=np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(1404)  
# 若frame裡面沒有出現左手,會跳error,要加if返回
lh=np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)  
rh=np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)  


In [ ]:
len(results.face_landmarks.landmark)*3

In [ ]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)  
    lh=np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)  
    rh=np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)  
    return np.concatenate([pose, face, lh, rh])

In [ ]:
extract_keypoints(results).shape

In [ ]:
len(results.left_hand_landmarks.landmark)

In [ ]:
lh.shape

In [ ]:
np.zeros(21*3)